# **Instructions**


*   Import your train and test datasets (as csv)
*   Update datasets path global variables
*   Play with model, epoch and batch size
*   Once you are happy with the result, turn 'saveModel' and 'convertModel' to 'True'

---


Dataset sample :  
```
Orientation,X,Y,Z,Hand
1,0.004,-0.680,-0.733,2
5,0.036,-0.660,-0.751,0
1,0.059,-0.655,-0.753,2
1,0.126,-0.725,-0.678,1
1,0.159,-0.710,-0.686,1
```
> Orientation values :  
1 = Portrait  
2 = PortraitUpsideDown  
3 = LandscapeLeft  
4 = LandscapeRight  
5 = FaceUp  
6 = FaceDown


---


> X, Y and Z values :  
float between -1 and 1


---


> Hand values :  
0 = Left  
1 = Right  
2 = Both

# **Pip install**

In [ ]:
pip install tf2onnx

In [ ]:
pip install plotly-express

# **Global variables**

In [ ]:
debug = True #Display outputs
saveModel = False
convertModel = False
array_line_to_print = 5

#Keep '/content/'
train_dataset_path = '/content/data/TrainData.csv'
test_dataset_path = '/content/data/TestData.csv'

# **Imports**

In [ ]:
from numpy import argmax
from pandas import read_csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot

import tf2onnx
import plotly.express as px

# **Load dataset**

In [ ]:
#@title Dataset { vertical-output: true, display-mode: "both" }
df = read_csv(train_dataset_path, header=0)

if debug:
  print(df[:array_line_to_print])

In [ ]:
#@title Dataset visualization { vertical-output: true, form-width: "60%" }
if debug:
  fig = px.scatter_3d(df, x='X', y='Y', z='Z', 
                      color='Hand', symbol='Orientation',
                      opacity=0.7, height=800, width=800)
  fig.show()

# **Split features and labels**

In [ ]:
#@title Features & Labels { vertical-output: true, display-mode: "both" }
X, y = df.values[:, :-1], df.values[:, -1]

if debug:
  print(X[:array_line_to_print])
  print(y[:array_line_to_print])

# **Validate data type**

In [ ]:
X = X.astype('float32')

# **Encode strings to integer**

In [ ]:
#@title Encoded strings { vertical-output: true, display-mode: "both" }
y = LabelEncoder().fit_transform(y)

if debug:
  print(set(y))

# **Split into train and test datasets**

In [ ]:
#@title Datasets shapes { vertical-output: true, display-mode: "both" }
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

if debug:
  print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# **Determine the number of input features**

In [ ]:
#@title Number of features { vertical-output: true, display-mode: "both" }
n_features = X_train.shape[1]

if debug:
  print(n_features)

# **Define model**

In [ ]:
#@title Model summary { vertical-output: true, form-width: "40%", display-mode: "both" }
model = Sequential(name='Sequential')
model.add(Dense(20, name='Input', activation='relu', kernel_initializer='he_normal', input_shape=(n_features,)))
#model.add(BatchNormalization(name='Batch_normalization'))
model.add(Dense(30, name='Hidden_layer', activation='relu', kernel_initializer='he_normal'))
model.add(Dense(20, name='Hidden_layer_1', activation='relu', kernel_initializer='he_normal'))
model.add(Dense(10, name='Hidden_layer_2', activation='relu', kernel_initializer='he_normal'))
#model.add(Dropout(0.5))
model.add(Dense(3, name='Output', activation='softmax'))

if debug:
  #model.summary()
  plot_model(model, 'model.png', show_shapes=True)

# **Compile the model**

In [ ]:
#sgd = SGD(learning_rate=0.001, momentum=0.8)
#model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# **Configure early stopping**

In [ ]:
es = EarlyStopping(monitor='loss', patience=5)

# **Fit the model**

In [ ]:
#@title Training { vertical-output: true, form-width: "30%", display-mode: "both" }
history = model.fit(X_train, y_train, epochs=150, batch_size=32, verbose=0, validation_split=0.3, callbacks=[es])



# **Evaluate the model**

In [ ]:
#@title Evaluation { vertical-output: true, display-mode: "both" }
loss, acc = model.evaluate(X_test, y_test, verbose=1)

# **Make a prediction**

In [ ]:
#@title Prediction { vertical-output: true, display-mode: "both" }
test_df = read_csv(test_dataset_path, header=0)

index = 0
class_to_predict = int(test_df.values[index, -1])

row = list(test_df.values[index, :-1])
print(row)
yhat = model.predict([row])

correct = class_to_predict == argmax(yhat)

print('Predicted: %d -> %s' % (argmax(yhat), correct))

# **Save model**

In [ ]:
if saveModel:
  model.save('/save/Models/mymodel')

# **Convert saved model to onnx format**

In [ ]:
if convertModel:
  !python -m tf2onnx.convert \
          --saved-model /content/Save/Models/mymodel/ \
          --output /content/Save/Models/handprediction.onnx \
          --opset 9